# AirBNB web scraping

### Install BeautifulSoap library

In [1]:
!pip install beautifulsoup4

### getting page link

In [60]:
import requests
import pandas as pd

In [61]:
url = "https://www.airbnb.co.in/s/Pune/homes?place_id=ChIJQ97RPE_AwjsR5zbDDbo3wHI&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=FLEXIBLE_DATES&search_type=HOMEPAGE_CAROUSEL_CLICK"

In [62]:
r = requests.get(url)
print(r)

<Response [200]>


In [63]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, "lxml")

### getting next page link

In [64]:
next_page = soup.find("a", {"aria-label" : "Next"}).get("href")
cnp = "https://www.airbnb.co.in/" + next_page
print(cnp)

https://www.airbnb.co.in//s/Pune/homes?place_id=ChIJQ97RPE_AwjsR5zbDDbo3wHI&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=flexible_dates&search_type=HOMEPAGE_CAROUSEL_CLICK&query=Pune&monthly_start_date=2025-07-01&monthly_length=3&monthly_end_date=2025-10-01&search_mode=regular_search&price_filter_input_type=2&price_filter_num_nights=2&channel=EXPLORE&federated_search_session_id=5649f41b-1f02-468f-a65e-0b064bba4863&pagination_search=true&cursor=eyJzZWN0aW9uX29mZnNldCI6MCwiaXRlbXNfb2Zmc2V0IjoxOCwidmVyc2lvbiI6MX0%3D


### creating empty list to store values

In [65]:
name_list = []
price_list = []
review_list = []

### Getting data from the website

In [66]:
try:
    while True:
        # Extract all names (find_all instead of find)
        names = soup.find_all("div", {
            "class": "t1jojoys atm_g3_1kw7nm4 atm_ks_15vqwwr atm_sq_1l2sidv atm_9s_cj1kg8 atm_6w_1e54zos atm_fy_1vgr820 atm_7l_jt7fhx atm_cs_10d11i2 atm_w4_1eetg7c atm_ks_zryt35__1rgatj2 dir dir-ltr"
        })
        for i in names:
            name_list.append(i.get_text(strip=True))

        # Extract all reviews
        review = soup.find_all("span", {"class": "a8jt5op atm_3f_idpfg4 atm_7h_hxbz6r atm_7i_ysn8ba atm_e2_t94yts atm_ks_zryt35 atm_l8_idpfg4 atm_vv_1q9ccgz atm_vy_t94yts au0q88m atm_mk_stnw88 atm_tk_idpfg4 dir dir-ltr"})  # update this if necessary
        for i in review:
            review_list.append(i.get_text(strip=True))

        # Extract all prices
        price = soup.find_all("span", {"class": "a8jt5op atm_3f_idpfg4 atm_7h_hxbz6r atm_7i_ysn8ba atm_e2_t94yts atm_ks_zryt35 atm_l8_idpfg4 atm_vv_1q9ccgz atm_vy_t94yts a1fdgz01 atm_mk_stnw88 atm_tk_idpfg4 dir dir-ltr"})  # update this if necessary
        for i in price:
            price_list.append(i.get_text(strip=True))

        # Find the next page link
        next_button = soup.find("a", {"aria-label": "Next"})
        if not next_button:
            break  # No more pages

        next_page = next_button.get("href")
        cnp = "https://www.airbnb.co.in" + next_page

        # Fetch next page
        r = requests.get(cnp)
        soup = BeautifulSoup(r.text, "lxml")

except Exception as e:
    print("Error occurred:", e)
print("Name: ", len(name_list))
print("price: ", len(price_list))
print("review: ", len(review_list))

Name:  270
price:  238
review:  269


In [67]:
df = pd.DataFrame({"Name" : pd.Series(name_list), "Review" : pd.Series(review_list), "Price" : pd.Series(price_list)})
df

,Name,Review,Price
0,Flat in Gahunje,"4.9 out of 5 average rating, 121 reviews","₹6,149 for 2 nights"
1,Apartment in Pune,"5.0 out of 5 average rating, 4 reviews","₹5,793 for 2 nights"
2,Flat in Pune,"4.9 out of 5 average rating, 49 reviews","₹4,793 for 2 nights"
3,Flat in Pune,"5.0 out of 5 average rating, 10 reviews","₹4,109 for 2 nights"
4,Guest suite in Pune,"4.79 out of 5 average rating, 67 reviews","₹5,033 for 2 nights"
...,...,...,...
265,Flat in Pimpri-Chinchwad,"4.91 out of 5 average rating, 32 reviews",NaN
266,Flat in Pune,"4.81 out of 5 average rating, 47 reviews",NaN
267,Cabin in Nadhal,"4.96 out of 5 average rating, 26 reviews",NaN
268,Flat in Pune,"4.92 out of 5 average rating, 331 reviews",NaN


In [70]:
df.fillna("NA")

,Name,Review,Price
0,Flat in Gahunje,"4.9 out of 5 average rating, 121 reviews","₹6,149 for 2 nights"
1,Apartment in Pune,"5.0 out of 5 average rating, 4 reviews","₹5,793 for 2 nights"
2,Flat in Pune,"4.9 out of 5 average rating, 49 reviews","₹4,793 for 2 nights"
3,Flat in Pune,"5.0 out of 5 average rating, 10 reviews","₹4,109 for 2 nights"
4,Guest suite in Pune,"4.79 out of 5 average rating, 67 reviews","₹5,033 for 2 nights"
...,...,...,...
265,Flat in Pimpri-Chinchwad,"4.91 out of 5 average rating, 32 reviews",NA
266,Flat in Pune,"4.81 out of 5 average rating, 47 reviews",NA
267,Cabin in Nadhal,"4.96 out of 5 average rating, 26 reviews",NA
268,Flat in Pune,"4.92 out of 5 average rating, 331 reviews",NA


In [72]:
df.to_csv("data.csv")